# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import pandas as pd
import hvplot.pandas
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,19.85,67,26,0.89,AU,1702335030
1,1,pregradnaya,43.9508,41.1994,7.61,64,100,1.16,RU,1702335031
2,2,petropavlovsk-kamchatsky,53.0452,158.6483,-9.67,87,40,7.60,RU,1702335033
3,3,polyarnyy,69.1989,33.4478,-12.96,91,96,6.91,RU,1702335034
4,4,olonkinbyen,70.9221,-8.7187,-0.06,98,100,6.30,SJ,1702335035


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

import geoviews
# Configure the map plot
import plotly.express as px

city_map = city_data_df.hvplot.points('Lng', 'Lat', geo=True, color='red', alpha=0.2, tiles='ESRI',frame_width = 700,
    frame_height = 500, size = "Population", legend=True,
    scale = .01)

# fig = px.scatter_mapbox(city_data_df, lat="Lat", lon="Lng", hover_name="City", hover_data=["City", "Humidity"],
#                         color_discrete_sequence=["fuchsia"], zoom=3, height=300)
# fig.update_layout(mapbox_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
clean_city_df = city_data_df.dropna()

# Drop any rows with null values
clean_city_df = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] >21) &
                                  (city_data_df['Wind Speed'] <= 4.5) & (city_data_df['Cloudiness'] ==0)]

# Display sample data
clean_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
133,133,macau,22.2006,113.5461,23.17,94,0,3.09,MO,1702335186
177,177,carnarvon,-24.8667,113.6333,26.17,31,0,2.88,AU,1702335235
319,319,vilhena,-12.7406,-60.1458,23.12,100,0,3.60,BR,1702335408
334,334,port hedland,-20.3167,118.5667,23.03,60,0,0.00,AU,1702335425
380,380,sultanah,24.4926,39.5857,22.25,16,0,2.57,SA,1702335484
400,400,tshabong,-26.0500,22.4500,22.01,21,0,4.19,BW,1702335512
418,418,isla vista,34.4133,-119.8610,24.99,38,0,3.09,US,1702335236
421,421,san marcos,33.1434,-117.1661,25.50,30,0,1.79,US,1702335539
467,467,toma,12.7667,-2.8833,21.80,17,0,4.25,BF,1702335596


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_df.filter(['City','Country','Lat','Lng','Humidity'], axis=1)
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
133,macau,MO,22.2006,113.5461,94,
177,carnarvon,AU,-24.8667,113.6333,31,
319,vilhena,BR,-12.7406,-60.1458,100,
334,port hedland,AU,-20.3167,118.5667,60,
380,sultanah,SA,24.4926,39.5857,16,
400,tshabong,BW,-26.0500,22.4500,21,
418,isla vista,US,34.4133,-119.8610,38,
421,san marcos,US,33.1434,-117.1661,30,
467,toma,BF,12.7667,-2.8833,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {"filter": "",
          "bias": "",
          "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index,'Lat']
    lng = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"&circle: {lng},{lat}"
    params["bias"] = f"&proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?categories=accommodation.hotel"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wailua homesteads - nearest hotel: No hotel found
kapa'a - nearest hotel: No hotel found
tomatlan - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
kedougou - nearest hotel: No hotel found
vallenar - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
senillosa - nearest hotel: No hotel found
les anglais - nearest hotel: No hotel found
frontera - nearest hotel: No hotel found
san rafael - nearest hotel: No hotel found
valparaiso - nearest hotel: No hotel found
sultanah - nearest hotel: No hotel found
al hawatah - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
ixtapa - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
38,wailua homesteads,US,22.0669,-159.3780,77,No hotel found
57,kapa'a,US,22.0752,-159.3190,77,No hotel found
82,tomatlan,MX,19.9333,-105.2500,70,No hotel found
105,lihue,US,21.9789,-159.3672,72,No hotel found
167,cabo san lucas,MX,22.8909,-109.9124,70,No hotel found
223,coahuayana de hidalgo,MX,18.7000,-103.6583,77,No hotel found
246,kedougou,SN,12.5579,-12.1743,41,No hotel found
276,vallenar,CL,-28.5708,-70.7581,39,No hotel found
280,guerrero negro,MX,27.9769,-114.0611,42,No hotel found
337,senillosa,AR,-39.0000,-68.4167,16,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng', 'Lat', geo=True, color='red', alpha=0.2, tiles='ESRI', frame_width = 700, 
frame_height = 500, legend=True, hover_cols=['Hotel Name','Lng','Lat','City','Country','Humidity'])
# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Country,Humidity)